In [1]:
import sys
sys.path.insert(0, '../')

from interface import get_query_emb_batch, load_model, cosine_similarity_batch

from typing import Tuple, List, Dict, Union, Any

from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import torch

In [2]:
csv_models = '/home/sondors/Documents/price/BERT_data/data/models_17_categories.csv'
df_models = pd.read_csv(csv_models, sep=';')

csv_offers = '/home/sondors/Documents/price/offers_clusterization/data/14_categories_balanced.csv'
df_offers = pd.read_csv(csv_offers, sep=';')

id_category = {
    3902: 'диктофоны, портативные рекордеры',
    510402: 'электронные книги',
    4302: 'автомобильные телевизоры, мониторы',
    2815: 'смарт-часы и браслеты',
    3901: 'портативные медиаплееры',
    3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    510401: 'планшетные компьютеры и мини-планшеты',
    # 2102: 'наушники, гарнитуры, наушники c микрофоном',
    3903: 'радиоприемники, радиобудильники, радиочасы',
    3907: 'магнитолы',
    280801: 'GPS-навигаторы'
    }

df_models = df_models[df_models['category_id'].isin(id_category.keys())].reset_index(drop=True)
df_offers = df_offers[df_offers['category_id'].isin(id_category.keys())].reset_index(drop=True)

models = list(df_models['full_name'])
offers = list(df_offers['name'])

df_offers

,price,name,attrs,description,model_id,external_category,external_brand,external_type,category_name,category_id
0,1890.0,Очки шлем виртуальной реальности с наушниками ...,"[{'id': 23412, 'type': 'e', 'value': 63357.0},...",Это самое лучшее предложение на существующем р...,3025579,Виртуальная реальность,NaN,NaN,"VR-гарнитуры (VR-очки, шлемы, очки виртуальной...",3908
1,2500.0,BoboVR Z4,"[{'id': 23412, 'type': 'e', 'value': 63356.0},...",Очки виртуальной реальности для смартфонов с в...,3025579,Очки виртуальной реальности :: Bobovr,BoboVR,NaN,"VR-гарнитуры (VR-очки, шлемы, очки виртуальной...",3908
2,2845.0,Очки виртуальной реальности BoboVR Z4,"[{'id': 23412, 'type': 'e', 'value': 63356.0}]",Описание товара Виртуальная реальность реальна...,3025579,Все товары :: Электроника :: Портативная техни...,BOBOVR,NaN,"VR-гарнитуры (VR-очки, шлемы, очки виртуальной...",3908
3,2299.0,Очки виртуальной реальности BOBOVR Z4 / Белые,"[{'id': 23412, 'type': 'e', 'value': 63356.0}]",NaN,3025579,Все товары :: Электроника :: Портативная техни...,BOBOVR,Очки виртуальной реальности,"VR-гарнитуры (VR-очки, шлемы, очки виртуальной...",3908
4,3800.0,BoboVR Очки BoboVr Z4 + джойстик/пульт управле...,"[{'id': 23412, 'type': 'e', 'value': 63356.0},...",600003652857,3025579,Электроника :: Компьютеры и комплектующие :: А...,BOBOVR,NaN,"VR-гарнитуры (VR-очки, шлемы, очки виртуальной...",3908
...,...,...,...,...,...,...,...,...,...,...
90805,49112.0,Sony Xperia 5 III 8/256Gb Pink,"[{'id': 4531, 'type': 'r', 'value': 8.0}, {'id...",Sony Xperia 5 III - компактный флагман в узком...,6372772,Все товары :: Электроника :: Телефоны :: Мобил...,Sony,NaN,мобильные телефоны,2801
90806,57672.0,Смартфон Sony Xperia 5 III 8/256Gb Black,"[{'id': 4531, 'type': 'r', 'value': 8.0}, {'id...",Sony Xperia 5 III - компактный флагман в узком...,6372772,Все товары :: Электроника :: Телефоны :: Мобил...,Sony,Смартфон,мобильные телефоны,2801
90807,47750.0,Смартфон Sony Xperia 5 III 8/256GB Green,"[{'id': 4531, 'type': 'r', 'value': 8.0}, {'id...","экран: 6.1"" (2520x1080) 120 Гцвстроенная памят...",6372772,Все товары :: Электроника :: Телефоны :: Мобил...,Sony,Смартфон,мобильные телефоны,2801
90808,84990.0,Смартфон Sony Xperia 5 III зелёный 256Gb,"[{'id': 4531, 'type': 'r', 'value': 8.0}, {'id...","Экран 6,1? 2520x1080, 5G, LTE, Встроенная памя...",6372772,Смартфоны XPERIA :: Xperia 5 III,Sony,Смартфон,мобильные телефоны,2801


In [3]:
df_models

,model_id,name,brand_name,full_name,category_name,category_id
0,331691,UP,Jawbone,Jawbone UP,смарт-часы и браслеты,2815
1,408055,Forerunner 10,Garmin,Garmin Forerunner 10,смарт-часы и браслеты,2815
2,473204,Pulse,Withings,Withings Pulse,смарт-часы и браслеты,2815
3,480007,SmartWatch 2 SW2,Sony,Sony SmartWatch 2 SW2,смарт-часы и браслеты,2815
4,484556,Galaxy Gear,Samsung,Samsung Galaxy Gear,смарт-часы и браслеты,2815
...,...,...,...,...,...,...
26315,7049424,MD-108,Mivo,Mivo MD-108,портативная акустика,3904
26316,7049425,MD-165,Mivo,Mivo MD-165,портативная акустика,3904
26317,7049426,Boost 20W,Rocket,Rocket Boost 20W,портативная акустика,3904
26318,7049427,Motion 10W,Rocket,Rocket Motion 10W,портативная акустика,3904


In [4]:
# ckpt_pth = "/home/sondors/Documents/ColBERT_weights/triples_X1_13_categories_use_ib_negatives/none/2024-01/26/10.49.44/checkpoints/colbert-5387-finish"
# experiment = "colbert-5387"

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/bert-base-multilingual-cased_dim_768_bsize_230_lr04_use_ib_negatives/none/2024-01/27/16.55.29/checkpoints/colbert-2998-finish"
experiment = "colbert-2998-finish"

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию. 

device = "cuda"


In [5]:
checkpoint = load_model(ckpt_pth, doc_maxlen, nbits, kmeans_niters, device)

In [6]:
offer_embs = get_query_emb_batch(offers, checkpoint, batch_size=1000, batch_size2=10000)

batch: 10000/90810
batch: 20000/90810
batch: 30000/90810
batch: 40000/90810
batch: 50000/90810
batch: 60000/90810
batch: 70000/90810
batch: 80000/90810
batch: 90000/90810
batch: 90810/90810


In [7]:
len(offers)

90810

In [8]:
model_embs = get_query_emb_batch(models, checkpoint, batch_size=1000, batch_size2=10000)


batch: 10000/26320


batch: 20000/26320
batch: 26320/26320


In [12]:
def top_n_similar(offer_embs: List[np.ndarray], model_embs: List[np.ndarray], model_ids: List[int], batch_size: int = 1000, n: int = 5) -> List[Dict[str, Union[List[int], np.ndarray]]]:
    """
    Find the top N similar embeddings for each offer embedding.

    Args:
        offer_embs (List[np.ndarray]): List of offer embeddings.
        model_embs (List[np.ndarray]): List of model embeddings.
        model_ids (List[int]): List of model IDs corresponding to model embeddings.
        batch_size (int, optional): Batch size for chunking the offer embeddings. Defaults to 1000.
        n (int, optional): Number of top similar embeddings to retrieve. Defaults to 5.

    Returns:
        List[Dict[str, Union[List[int], np.ndarray]]]: A list of dictionaries containing model IDs and their 
        corresponding cosine similarity scores for the top N similar embeddings for each offer embedding.

    Example:
        >>> offer_embeddings = [np.array([0.1, 0.2, 0.3]), np.array([0.4, 0.5, 0.6])]
        >>> model_embeddings = [np.array([0.2, 0.3, 0.4]), np.array([0.5, 0.6, 0.7])]
        >>> model_ids = [123, 456]
        >>> print(top_n_similar(offer_embeddings, model_embeddings, model_ids))
            # Output: [{'model_ids': [123, 456], 'cosine_sims': array([0.99258333, 0.96832966])},
            {'model_ids': [456, 123], 'cosine_sims': array([0.99964575, 0.99461155])}]
    """

    cosine_sims = cosine_similarity_batch(offer_embs, model_embs, batch_size)
    top_n_list = []
    for i in range(len(offer_embs)):
        # Find the indices of the top N similar embeddings
        top_n_indices = np.argsort(cosine_sims[i])[::-1][:n]
        # Get the corresponding model IDs and cosine similarity scores for the top N similar embeddings
        top_n_model_ids = [model_ids[i] for i in top_n_indices]
        top_n_cosine_sims = cosine_sims[i][top_n_indices]
        top_n_list.append({"model_ids": top_n_model_ids, "cosine_sims": top_n_cosine_sims})
    return top_n_list

In [13]:
import time
    
def emb_to_df(df, embs):
    embed_name = []
    for index, row in df.iterrows():
        embed_name.append(embs[index])
    return embed_name

def calc_acc_for_category(df_offers, df_models, imya_kategorii, models_embs, offers_embs):
    print(f"{imya_kategorii}")
    start_time = time.time()
    df_offers = df_offers[df_offers.category_name == imya_kategorii]
    df_models = df_models[df_models.category_name == imya_kategorii]

    offer_embs = emb_to_df(df_offers, offers_embs)
    offer_model_id = list(df_offers['model_id'])

    model_embs = emb_to_df(df_models, models_embs)
    model_model_id = list(df_models['model_id'])

    print(f"\tофферов {len(offer_embs)} шт поиск матча для каждого среди {len(model_model_id)} шт моделей")

    batch_size = max((1, (5000*4000)//len(model_model_id)))
    top_n = top_n_similar(offer_embs, model_embs, model_model_id, batch_size, n=5)
    offer_model_id_predicted = [i['model_ids'][0] for i in top_n]

    equal = np.sum(np.array(offer_model_id) == np.array(offer_model_id_predicted))
    accuracy = round(100*equal/len(offer_model_id), 2)
    print(f"\taccuracy = {accuracy}%\n\ttime_spent = {round(time.time() - start_time, 2)} sec\n")
    return accuracy


accuracy_one_model = []
for category in id_category.values():
    accuracy = calc_acc_for_category(df_offers, df_models, category, model_embs, offer_embs)
    accuracy_one_model.append(accuracy)


диктофоны, портативные рекордеры
	офферов 710 шт поиск матча для каждого среди 488 шт моделей
	accuracy = 92.25%
	time_spent = 0.19 sec

электронные книги
	офферов 760 шт поиск матча для каждого среди 680 шт моделей
	accuracy = 96.97%
	time_spent = 0.22 sec

автомобильные телевизоры, мониторы
	офферов 280 шт поиск матча для каждого среди 790 шт моделей
	accuracy = 89.64%
	time_spent = 0.13 sec

смарт-часы и браслеты
	офферов 11210 шт поиск матча для каждого среди 2577 шт моделей
	accuracy = 93.42%
	time_spent = 3.64 sec

портативные медиаплееры
	офферов 1410 шт поиск матча для каждого среди 847 шт моделей
	accuracy = 93.69%
	time_spent = 0.29 sec

портативная акустика
	офферов 15610 шт поиск матча для каждого среди 3800 шт моделей
	accuracy = 95.9%
	time_spent = 7.04 sec

мобильные телефоны
	офферов 49490 шт поиск матча для каждого среди 9494 шт моделей
	accuracy = 96.0%
	time_spent = 56.04 sec

VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)
	оф